In [1]:
import os
os.chdir('C:/Users/RH/A_whole_try_prog/y_test_data')

In [2]:
import pandas as pd
import numpy as np
import esm
import torch

print(pd.__version__)
print(np.__version__)
print(esm.__version__)
print(torch.__version__)

1.5.3
1.22.0
2.0.0
2.0.0


In [3]:
import numpy as np
import pandas as pd

In [4]:
#training dataset loading
#打开指定路径的excel文件
excel_path = r'C:\Users\RH\A_whole_try_prog\X_test_nr.xlsx'
dataset = pd.read_excel(excel_path, na_filter=False)
sequence_list = dataset['Sequence']

In [5]:
print(sequence_list)

0                  AHK
1                  VHH
2              VHHANEN
3             VNPHDHQN
4            LVNPHDHQN
5           LLPHHADADY
6              VIPAGYP
7     LQSGDALRVPSGTTYY
8                  VKL
9                 VVKL
10                 IVF
11                 FSL
12                  PP
13              SVPQPK
14               EVPKA
15              PLAQPA
16               LLNPT
17               HLPLP
18              VLPIPQ
19                 LWL
Name: Sequence, dtype: object


In [6]:
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence)
print(peptide_sequence_list)

[('AHK', 'AHK'), ('VHH', 'VHH'), ('VHHANEN', 'VHHANEN'), ('VNPHDHQN', 'VNPHDHQN'), ('LVNPHDHQN', 'LVNPHDHQN'), ('LLPHHADADY', 'LLPHHADADY'), ('VIPAGYP', 'VIPAGYP'), ('LQSGDALRVPSGTTYY', 'LQSGDALRVPSGTTYY'), ('VKL', 'VKL'), ('VVKL', 'VVKL'), ('IVF', 'IVF'), ('FSL', 'FSL'), ('PP', 'PP'), ('SVPQPK', 'SVPQPK'), ('EVPKA', 'EVPKA'), ('PLAQPA', 'PLAQPA'), ('LLNPT', 'LLNPT'), ('HLPLP', 'HLPLP'), ('VLPIPQ', 'VLPIPQ'), ('LWL', 'LWL')]


In [7]:
#调试完毕，以此为准！
import torch
import esm
import pandas as pd

#load ESM-2 model
model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()
    
#load data
data = peptide_sequence_list
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[12], return_contacts=True)
token_representations = results["representations"][12]
    
# Generate per-sequence representations via averaging
sequence_representations = []
for i, token_len in enumerate(batch_lens):
    each_seq_rep = token_representations[i, 1:token_len - 1].mean(0).tolist()  
    sequence_representations.append(each_seq_rep)

embedding_results = pd.DataFrame(sequence_representations)
print(embedding_results)
embedding_results.to_csv('X_test_480.csv')

         0         1         2         3         4         5         6    \
0  -0.116332 -0.350103  0.233661  0.245473 -0.147785 -0.128958 -0.331269   
1  -0.054674 -0.235630  0.198577  0.241546 -0.169054 -0.155177 -0.392659   
2  -0.000828 -0.090696  0.206846  0.129747 -0.103830 -0.113478 -0.219809   
3  -0.041371 -0.112135  0.244645  0.181520 -0.093399 -0.163702 -0.276983   
4  -0.047274 -0.107493  0.220792  0.184172 -0.056658 -0.132734 -0.182958   
5   0.035068 -0.202087  0.184955  0.121891 -0.112658 -0.063387 -0.193854   
6   0.027588 -0.132473  0.233385  0.126259 -0.065988 -0.003404 -0.266010   
7   0.083690 -0.036041  0.027686 -0.070589  0.034067 -0.056349 -0.128763   
8  -0.183499 -0.015560  0.146414  0.282784  0.050134 -0.060260 -0.347426   
9  -0.130950  0.005240  0.199912  0.323561  0.072376 -0.002095 -0.375826   
10 -0.229531  0.266537  0.091407  0.491034  0.045132 -0.083081 -0.352996   
11 -0.326863  0.050528 -0.003625  0.270295  0.144629 -0.050793 -0.212074   
12 -0.038697